# Maintenance Score Implementation

This notebook concentrates on reimplementing the score "maintained". Explained on the documentation, maintainment score concentrates on the activity data over last 90 days.

In [2]:
# Importing the libraries
import pandas as pd

In [4]:
pi_df = pd.read_parquet('Data/Processed/Metrics/project_information.parquet')
pi_df

,01-2024,02-2024,03-2024,04-2024,05-2024,06-2024,07-2024
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
105,False,False,False,False,False,False,False
106,False,False,False,False,False,False,False
107,False,False,False,False,False,False,False
108,False,False,False,False,False,False,False


In [5]:
commits_df = pd.read_parquet('Data/Processed/Metrics/commit_per_month.parquet')
commits_df

,01-2024,02-2024,03-2024,04-2024,05-2024,06-2024,07-2024
0,0,0,0,3,0,0,0
1,0,0,0,0,7,0,0
2,2,1,2,0,0,9,0
3,0,0,0,0,0,0,0
4,77,125,99,122,116,98,97
...,...,...,...,...,...,...,...
105,0,0,0,0,1,0,0
106,0,6,2,0,0,5,14
107,4,3,0,2,2,2,2
108,3,2,0,0,0,2,0


In [6]:
issues_df = pd.read_parquet('Data/Processed/Metrics/issues.parquet')
issues_df

,01-2024,02-2024,03-2024,04-2024,05-2024,06-2024,07-2024
0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,5,1,0
4,2,4,7,3,4,1,2
...,...,...,...,...,...,...,...
105,0,0,0,0,2,0,0
106,0,0,0,0,0,0,0
107,0,1,0,0,0,0,0
108,9,3,0,0,0,1,0


## Predefined Functions and Initialization of Months

In [5]:
def calculate_three_month_score(df):
    score = df.sum(axis=1)
    return score

def check_all_true(df):
    return df.apply(lambda row: row[df.columns[0]] and row[df.columns[1]] and row[df.columns[2]], axis=1)

In [6]:
# Define the start and end dates (we are getting three months before of the starting date, because each month should consider the activities based on the last 90 days)
start_year, start_month = 2023, 1
end_year, end_month = 2023, 12

# Generate the list of months between start and end dates
months = pd.date_range(start=f"{start_month}-{start_year}", end=f"{end_month}-{end_year}", freq='MS').strftime("%m-%Y").tolist()

## Component Calculation

In [7]:
pi_activity_score = pd.DataFrame(index=pi_df.index, columns=months)
commit_activity_score = pd.DataFrame(index=commits_df.index, columns=months)
issue_activity_score = pd.DataFrame(index=issues_df.index, columns=months)

for i in range(len(pi_df.columns) - 3):
    pi_activity_score.iloc[:, i] = check_all_true(pi_df.iloc[:, i:i+3])
    commit_activity_score.iloc[:, i] = calculate_three_month_score(commits_df.iloc[:, i:i+3])
    issue_activity_score.iloc[:, i] = calculate_three_month_score(issues_df.iloc[:, i:i+3])

IndexError: iloc cannot enlarge its target object

In [ ]:
pi_activity_score

,01-2023,02-2023,03-2023,04-2023,05-2023,06-2023,07-2023,08-2023,09-2023,10-2023,11-2023,12-2023,01-2024,02-2024,03-2024,04-2024,05-2024,06-2024
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24775,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
24776,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
24777,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
24778,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


## Reimplementation of the Maintenance Score

In [ ]:
maintained_score = commit_activity_score + issue_activity_score
t = 4 * 90 / 30
maintained_score = maintained_score.map(lambda x: min(10 * x  / t, 10)) 
maintained_score = maintained_score.where(pi_activity_score, 0)
maintained_score = maintained_score.astype(int)
maintained_score

,01-2023,02-2023,03-2023,04-2023,05-2023,06-2023,07-2023,08-2023,09-2023,10-2023,11-2023,12-2023,01-2024,02-2024,03-2024,04-2024,05-2024,06-2024
0,10,10,4,0,0,5,5,5,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24775,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24776,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24777,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24778,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
maintained_score.to_parquet('Data/Processed/Metrics/maintenance_score.parquet')